### Importing Necessary Libraries

In [ ]:
#to hide warnings
import warnings
warnings.filterwarnings("ignore")

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix,classification_report,accuracy_score
from sklearn.linear_model import SGDClassifier
from sklearn.feature_selection import RFE
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier

### Reading data

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
#Creating a dataframe for train data
data = pd.read_csv("/content/drive/MyDrive/Hackathon/employee_train.csv",header = 0,index_col=None)
#Creating a dataframe for test data
test_data = pd.read_csv("/content/drive/MyDrive/Hackathon/employee_test.csv",header = 0,index_col=None)

In [ ]:
pd.set_option('display.max_columns',None)
data.head()

,S.No,Timestamp,Age,Gender,Country,state,self_employed,family_history,treatment,work_interfere,no_employees,remote_work,tech_company,benefits,care_options,wellness_program,seek_help,anonymity,leave,mental_health_consequence,phys_health_consequence,coworkers,supervisor,mental_health_interview,phys_health_interview,mental_vs_physical,obs_consequence,comments
0,1,8/27/2014 11:29,37,Female,United States,IL,NaN,No,Yes,Often,25-Jun,No,Yes,Yes,Not sure,No,Yes,Yes,Somewhat easy,No,No,Some of them,Yes,No,Maybe,Yes,No,NaN
1,2,8/27/2014 11:29,44,M,United States,IN,NaN,No,No,Rarely,More than 1000,No,No,Don't know,No,Don't know,Don't know,Don't know,Don't know,Maybe,No,No,No,No,No,Don't know,No,NaN
2,3,8/27/2014 11:29,32,Male,Canada,NaN,NaN,No,No,Rarely,25-Jun,No,Yes,No,No,No,No,Don't know,Somewhat difficult,No,No,Yes,Yes,Yes,Yes,No,No,NaN
3,4,8/27/2014 11:29,31,Male,United Kingdom,NaN,NaN,Yes,Yes,Often,26-100,No,Yes,No,Yes,No,No,No,Somewhat difficult,Yes,Yes,Some of them,No,Maybe,Maybe,No,Yes,NaN
4,5,8/27/2014 11:30,31,Male,United States,TX,NaN,No,No,Never,100-500,Yes,Yes,Yes,No,Don't know,Don't know,Don't know,Don't know,No,No,Some of them,Yes,Yes,Yes,Don't know,No,NaN


In [ ]:
test_data.head()

,S.No,Timestamp,Age,Gender,Country,state,self_employed,family_history,work_interfere,no_employees,remote_work,tech_company,benefits,care_options,wellness_program,seek_help,anonymity,leave,mental_health_consequence,phys_health_consequence,coworkers,supervisor,mental_health_interview,phys_health_interview,mental_vs_physical,obs_consequence,comments
0,1,8/29/2014 11:32,39,Male,United Kingdom,NaN,Yes,Yes,Sometimes,5-Jan,Yes,Yes,No,Yes,Yes,Yes,Yes,Somewhat difficult,No,No,Yes,Yes,No,Maybe,Yes,Yes,These result may be a tad confusing so a summa...
1,2,8/29/2014 11:32,26,female,United States,WA,No,Yes,Sometimes,More than 1000,No,Yes,Yes,Yes,No,No,Don't know,Don't know,No,No,Some of them,Yes,No,Maybe,No,Yes,I should note one of the places my employer fa...
2,3,8/29/2014 11:33,23,Female,United States,IL,No,Yes,Sometimes,26-100,No,No,No,No,No,No,Don't know,Somewhat difficult,Yes,No,No,Some of them,No,Maybe,No,No,NaN
3,4,8/29/2014 11:34,35,Male,Switzerland,NaN,No,Yes,Often,More than 1000,No,Yes,Don't know,Not sure,No,No,Yes,Very easy,No,No,Some of them,Some of them,No,Maybe,No,No,NaN
4,5,8/29/2014 11:36,36,Male,United States,FL,No,No,Never,5-Jan,Yes,Yes,Don't know,Not sure,Don't know,Don't know,Don't know,Very easy,No,No,Some of them,Some of them,No,No,Don't know,No,NaN


### Understanding the data

In [ ]:
data.shape

(1048, 28)

In [ ]:
data.columns

Index(['S.No', 'Timestamp', 'Age', 'Gender', 'Country', 'state',
       'self_employed', 'family_history', 'treatment', 'work_interfere',
       'no_employees', 'remote_work', 'tech_company', 'benefits',
       'care_options', 'wellness_program', 'seek_help', 'anonymity', 'leave',
       'mental_health_consequence', 'phys_health_consequence', 'coworkers',
       'supervisor', 'mental_health_interview', 'phys_health_interview',
       'mental_vs_physical', 'obs_consequence', 'comments'],
      dtype='object')

In [ ]:
data.dtypes

,0
S.No,int64
Timestamp,object
Age,int64
Gender,object
Country,object
state,object
self_employed,object
family_history,object
treatment,object
work_interfere,object


In [ ]:

data.describe(include = 'all')

,S.No,Timestamp,Age,Gender,Country,state,self_employed,family_history,treatment,work_interfere,no_employees,remote_work,tech_company,benefits,care_options,wellness_program,seek_help,anonymity,leave,mental_health_consequence,phys_health_consequence,coworkers,supervisor,mental_health_interview,phys_health_interview,mental_vs_physical,obs_consequence,comments
count,1048.000000,1048,1.048000e+03,1048,1048,636,1030,1048,1048,812,1048,1048,1048,1035,1048,1044,1048,1048,1044,1048,1048,1048,1048,1048,1048,1048,1048,132
unique,NaN,679,NaN,45,42,45,2,2,2,4,6,2,2,3,3,3,3,3,5,3,3,3,3,3,3,3,2,128
top,NaN,8/27/2014 12:31,NaN,Male,United States,CA,No,No,No,Sometimes,25-Jun,No,Yes,Yes,No,No,No,Don't know,Don't know,No,No,Some of them,Yes,No,Maybe,Don't know,No,* Small family business - YMMV.
freq,NaN,8,NaN,518,644,123,906,643,536,386,245,733,870,400,422,692,534,685,462,424,773,651,439,835,461,471,902,5
mean,524.500000,NaN,9.541988e+07,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
std,302.675844,NaN,3.089010e+09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
min,1.000000,NaN,-1.726000e+03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25%,262.750000,NaN,2.700000e+01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
50%,524.500000,NaN,3.100000e+01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
75%,786.250000,NaN,3.600000e+01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1048 entries, 0 to 1047
Data columns (total 28 columns):
 #   Column                     Non-Null Count  Dtype 
---  ------                     --------------  ----- 
 0   S.No                       1048 non-null   int64 
 1   Timestamp                  1048 non-null   object
 2   Age                        1048 non-null   int64 
 3   Gender                     1048 non-null   object
 4   Country                    1048 non-null   object
 5   state                      636 non-null    object
 6   self_employed              1030 non-null   object
 7   family_history             1048 non-null   object
 8   treatment                  1048 non-null   object
 9   work_interfere             812 non-null    object
 10  no_employees               1048 non-null   object
 11  remote_work                1048 non-null   object
 12  tech_company               1048 non-null   object
 13  benefits                   1035 non-null   object
 14  care_opt

In [ ]:
for i in data.columns:
    if i != 'Timestamp' and i != 'comments':
        print("COLUMN: ", i.upper())
        print("")
        print(data[i].unique())
        print("")

COLUMN:  S.NO

[   1    2    3 ... 1046 1047 1048]

COLUMN:  AGE

[         37          44          32          31          33          35
          39          42          23          29          36          27
          46          41          34          30          40          38
          50          24          18          28          26          22
          19          25          45          21         -29          43
          56          60          54         329          55 99999999999
          48          20          57          58          47          62
          51          65          49       -1726           5          53
          61           8]

COLUMN:  GENDER

['Female' 'M' 'Male' 'male' 'female' 'm' 'Male-ish' 'maile' 'Trans-female'
 'Cis Female' 'F' 'something kinda male?' 'Cis Male' 'Woman' 'f' 'Mal'
 'Male (CIS)' 'queer/she/they' 'non-binary' 'Femake' 'woman' 'Make' 'Nah'
 'All' 'Enby' 'fluid' 'Genderqueer' 'Female ' 'Androgyne' 'Agender'
 'cis-female/femme

## Data Preprocessing

### 1). Feature Selection

In [ ]:
#we'll drop the data from the copy of dataframe to keep the original data intact
df = data.drop(['S.No', 'Timestamp','Country', 'state','comments'], axis =1)

df_test = test_data.drop(['S.No', 'Timestamp','Country', 'state','comments'], axis =1)

In [ ]:
df.head()

,Age,Gender,self_employed,family_history,treatment,work_interfere,no_employees,remote_work,tech_company,benefits,care_options,wellness_program,seek_help,anonymity,leave,mental_health_consequence,phys_health_consequence,coworkers,supervisor,mental_health_interview,phys_health_interview,mental_vs_physical,obs_consequence
0,37,Female,NaN,No,Yes,Often,25-Jun,No,Yes,Yes,Not sure,No,Yes,Yes,Somewhat easy,No,No,Some of them,Yes,No,Maybe,Yes,No
1,44,M,NaN,No,No,Rarely,More than 1000,No,No,Don't know,No,Don't know,Don't know,Don't know,Don't know,Maybe,No,No,No,No,No,Don't know,No
2,32,Male,NaN,No,No,Rarely,25-Jun,No,Yes,No,No,No,No,Don't know,Somewhat difficult,No,No,Yes,Yes,Yes,Yes,No,No
3,31,Male,NaN,Yes,Yes,Often,26-100,No,Yes,No,Yes,No,No,No,Somewhat difficult,Yes,Yes,Some of them,No,Maybe,Maybe,No,Yes
4,31,Male,NaN,No,No,Never,100-500,Yes,Yes,Yes,No,Don't know,Don't know,Don't know,Don't know,No,No,Some of them,Yes,Yes,Yes,Don't know,No


In [ ]:
df_test.head()

,Age,Gender,self_employed,family_history,work_interfere,no_employees,remote_work,tech_company,benefits,care_options,wellness_program,seek_help,anonymity,leave,mental_health_consequence,phys_health_consequence,coworkers,supervisor,mental_health_interview,phys_health_interview,mental_vs_physical,obs_consequence
0,39,Male,Yes,Yes,Sometimes,5-Jan,Yes,Yes,No,Yes,Yes,Yes,Yes,Somewhat difficult,No,No,Yes,Yes,No,Maybe,Yes,Yes
1,26,female,No,Yes,Sometimes,More than 1000,No,Yes,Yes,Yes,No,No,Don't know,Don't know,No,No,Some of them,Yes,No,Maybe,No,Yes
2,23,Female,No,Yes,Sometimes,26-100,No,No,No,No,No,No,Don't know,Somewhat difficult,Yes,No,No,Some of them,No,Maybe,No,No
3,35,Male,No,Yes,Often,More than 1000,No,Yes,Don't know,Not sure,No,No,Yes,Very easy,No,No,Some of them,Some of them,No,Maybe,No,No
4,36,Male,No,No,Never,5-Jan,Yes,Yes,Don't know,Not sure,Don't know,Don't know,Don't know,Very easy,No,No,Some of them,Some of them,No,No,Don't know,No


### 2). Handling Missing Values

In [ ]:
#Check for missing values count in whole dataset
df.isnull().sum()

,0
Age,0
Gender,0
self_employed,18
family_history,0
treatment,0
work_interfere,236
no_employees,0
remote_work,0
tech_company,0
benefits,13


In [ ]:
df_test.isnull().sum()

,0
Age,0
Gender,0
self_employed,0
family_history,0
work_interfere,28
no_employees,0
remote_work,0
tech_company,0
benefits,3
care_options,0


In [ ]:
#we can use use fillna to fill the missing values with mean of the variable
#inplace is kept as true to work om only the missing values rather than the whole data and thereby reducing kernel load
df['self_employed'].fillna(df['self_employed'].mode()[0], inplace = True)

df['benefits'].fillna(df['benefits'].mode()[0], inplace = True)

df['wellness_program'].fillna(df['wellness_program'].mode()[0], inplace = True)

df['leave'].fillna(df['leave'].mode()[0], inplace = True)

In [ ]:
df['work_interfere'].fillna("Don't Know", inplace = True)

In [ ]:
df_test.isnull().sum()

,0
Age,0
Gender,0
self_employed,0
family_history,0
work_interfere,28
no_employees,0
remote_work,0
tech_company,0
benefits,3
care_options,0


In [ ]:
df_test['benefits'].fillna(df_test['benefits'].mode()[0], inplace = True)

df_test['wellness_program'].fillna(df_test['wellness_program'].mode()[0], inplace = True)

df_test['leave'].fillna(df_test['leave'].mode()[0], inplace = True)

In [ ]:
df_test['work_interfere'].fillna("Don't Know", inplace = True)

In [ ]:
df_test.isnull().sum()

,0
Age,0
Gender,0
self_employed,0
family_history,0
work_interfere,0
no_employees,0
remote_work,0
tech_company,0
benefits,0
care_options,0


### Handling Age variable

In [ ]:
#calculating mean for values between 1 to 100
mean_age = int(df['Age'][(df['Age']<=100) & (df['Age']>=1)].mean())
mean_age_test = int(df_test['Age'][(df_test['Age']<=100) & (df_test['Age']>=1)].mean())

In [ ]:
#Replacing values greater than 65 or lower than 17 with mean
df['Age'] = df['Age'].apply(lambda x: mean_age if (x>100 or x<1) else int(x))
df_test['Age']= df_test['Age'].apply(lambda x: mean_age_test if(x>100 or x<1) else int(x))

### 3). Converting Categorical/Discrete Var to Continuous Var

### 1.) Gender

In [ ]:
df['Gender'] = df['Gender'].replace(("female", "f", "woman", "femake", "cis female", "female (cis)", "female (trans)", "trans woman"),0)
df['Gender'] = df['Gender'].replace(("male", "m", "man", "guy", "mal", "mail", "msle", "maile", "malr", "male-ish", "cis male", "male (cis)"),1)
df['Gender'] = df['Gender'].replace(('Trans-female','something kinda male?','queer/she/they','non-binary',
                                 'Nah','All','Enby','fluid','Genderqueer','Androgyne','Agender','Guy (-ish) ^_^',
                                  'male leaning androgynous','Trans woman','Neuter','Female (trans)','queer',
                                  'A little about you'),2)

In [ ]:
# to check for the unique values of gender in test data
df_test['Gender'].unique()

array(['Male', 'female', 'Female', 'M', 'male', 'Male ', 'm', 'p', 'F',
       'Woman', 'femail', 'Cis Man',
       'ostensibly male, unsure what that really means', 'f'],
      dtype=object)

In [ ]:
#Applying encoding to test data
df_test['Gender'] = df_test['Gender'].replace(('Female','female','F','Woman','f','femail'),0)
df_test['Gender'] = df_test['Gender'].replace(('M','Male','male','m','Male ','Cis Man'),1)
df_test['Gender'] = df_test['Gender'].replace(('p','ostensibly male, unsure what that really means'),2)

### 2.) no_employees

In [ ]:
def standardize_no_employees(value):
    """Converts inconsistent employee range formats into standardized ranges."""
    value = str(value).strip().lower()  # Convert to lowercase & remove spaces

    # Define mappings
    if value in ["5-jan", "jan-5", "1-5"]:
        return "1-5"
    elif value in ["25-jun", "jun-25", "6-25"]:
        return "6-25"
    elif value == "26-100":
        return "26-100"
    elif value == "100-500":
        return "100-500"
    elif value == "500-1000":
        return "500-1000"
    elif value == "more than 1000":
        return "1000+"
    else:
        return "Unknown"  # Handle unexpected values

In [ ]:
# Apply function to train & test datasets
df["no_employees"] = df["no_employees"].apply(standardize_no_employees)
df_test["no_employees"] = df_test["no_employees"].apply(standardize_no_employees)

# Check unique values after transformation
print("🔹 Train Dataset Employee Count Categories:\n", df["no_employees"].value_counts())
print("\n🔹 Test Dataset Employee Count Categories:\n", df_test["no_employees"].value_counts())

🔹 Train Dataset Employee Count Categories:
 no_employees
6-25        245
26-100      238
1000+       227
100-500     150
1-5         133
500-1000     55
Name: count, dtype: int64

🔹 Test Dataset Employee Count Categories:
 no_employees
1000+       55
26-100      51
6-25        45
1-5         28
100-500     26
500-1000     5
Name: count, dtype: int64


In [ ]:
df_test.dtypes

,0
Age,int64
Gender,int64
self_employed,object
family_history,object
work_interfere,object
no_employees,object
remote_work,object
tech_company,object
benefits,object
care_options,object


In [ ]:
#Reducing redundant class to a single class for 'leaves' variable
df['leave'] = df['leave'].replace(['Somewhat easy', 'Very easy'],'easy')
df_test['leave'] = df_test['leave'].replace(['Somewhat easy', 'Very easy'],'easy')

In [ ]:
encode = LabelEncoder()

for i in df.columns:
    if df[i].dtype == 'object':  # Check if the column is categorical
        df[i] = df[i].astype(str)  # Convert all values to strings to avoid type errors
        df[i] = encode.fit_transform(df[i])

In [ ]:
#To check if all the data types have been converted to int from object
df.dtypes

,0
Age,int64
Gender,int64
self_employed,int64
family_history,int64
treatment,int64
work_interfere,int64
no_employees,int64
remote_work,int64
tech_company,int64
benefits,int64


In [ ]:
df.head()

,Age,Gender,self_employed,family_history,treatment,work_interfere,no_employees,remote_work,tech_company,benefits,care_options,wellness_program,seek_help,anonymity,leave,mental_health_consequence,phys_health_consequence,coworkers,supervisor,mental_health_interview,phys_health_interview,mental_vs_physical,obs_consequence
0,37,7,0,0,1,2,5,0,1,2,1,1,2,2,3,1,1,1,2,1,0,2,0
1,44,10,0,0,0,3,2,0,0,0,0,0,0,0,0,0,1,0,0,1,1,0,0
2,32,14,0,0,0,3,5,0,1,1,0,1,1,0,1,1,1,2,2,2,2,1,0
3,31,14,0,1,1,2,3,0,1,1,2,1,1,1,1,2,2,1,0,0,0,1,1
4,31,14,0,0,0,1,1,1,1,2,0,0,0,0,0,1,1,1,2,2,2,0,0


In [ ]:
#encoding variables for data from test set
for i in df_test.columns:
    if df_test[i].dtype == 'object':
            df_test[i] = encode.fit_transform(df_test[i])

In [ ]:
#To check if all the data types have been converted to int from object
df_test.dtypes

,0
Age,int64
Gender,int64
self_employed,int64
family_history,int64
work_interfere,int64
no_employees,int64
remote_work,int64
tech_company,int64
benefits,int64
care_options,int64


In [ ]:
df_test.head()

,Age,Gender,self_employed,family_history,work_interfere,no_employees,remote_work,tech_company,benefits,care_options,wellness_program,seek_help,anonymity,leave,mental_health_consequence,phys_health_consequence,coworkers,supervisor,mental_health_interview,phys_health_interview,mental_vs_physical,obs_consequence
0,39,1,1,1,4,0,1,1,1,2,2,2,2,1,1,1,2,2,1,0,2,1
1,26,0,0,1,4,2,0,1,2,2,1,1,0,0,1,1,1,2,1,0,1,1
2,23,0,0,1,4,3,0,0,1,0,1,1,0,1,2,1,0,1,1,0,1,0
3,35,1,0,1,2,2,0,1,0,1,1,1,2,3,1,1,1,1,1,0,1,0
4,36,1,0,0,1,0,1,1,0,1,0,0,0,3,1,1,1,1,1,1,0,0


### Creating x and y

In [ ]:
# Target/Dependent variable is Treatment

x = df.loc[:, df.columns != 'treatment']
y = df.loc[:,'treatment']

In [ ]:
print(x.shape)
print(y.shape)

(1048, 22)
(1048,)


### Scaling the data

In [ ]:
scaler = StandardScaler()
scaler.fit(x)
x = scaler.transform(x)

In [ ]:
scaler.fit(df_test)
df_test = scaler.transform(df_test)

### Splitting the data into train and test set

##### Splitting train set to train and validation set as we already have our test set

In [ ]:
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.3,random_state = 10,stratify = y)

In [ ]:
print("x_train:",x_train.shape)
print("x_test:",x_test.shape)
print("y_train:",y_train.shape)
print("y_test:",y_test.shape)

x_train: (733, 22)
x_test: (315, 22)
y_train: (733,)
y_test: (315,)


## Building Logistic Regression Model

In [ ]:
 #training the model
lr = LogisticRegression()
lr.fit(x_train,y_train)

LogisticRegression()

In [ ]:
#Looking for coefficients value of output equation as logistic regression outputs a equation too
Coefficients = list(zip(df.columns[:-1],lr.coef_.ravel())) #ravel() is used to flatten the array
print(Coefficients)
print(lr.intercept_)

[('Age', np.float64(0.14981940404327612)), ('Gender', np.float64(-0.10227944350272038)), ('self_employed', np.float64(-0.027047937466980646)), ('family_history', np.float64(0.5791734279416397)), ('treatment', np.float64(1.4659374398245488)), ('work_interfere', np.float64(0.11521040022704752)), ('no_employees', np.float64(0.02778250664147915)), ('remote_work', np.float64(-0.13332994593780642)), ('tech_company', np.float64(0.23963482104064462)), ('benefits', np.float64(0.3300003708996531)), ('care_options', np.float64(-0.05089846574444168)), ('wellness_program', np.float64(-0.16291418875713434)), ('seek_help', np.float64(0.3035531490767121)), ('anonymity', np.float64(-0.11210484451364865)), ('leave', np.float64(-0.02779464563179514)), ('mental_health_consequence', np.float64(0.08866852178221532)), ('phys_health_consequence', np.float64(0.2384642926218501)), ('coworkers', np.float64(-0.1564863822006485)), ('supervisor', np.float64(0.07511625241760363)), ('mental_health_interview', np.floa

In [ ]:
#outputting Probability matrix
prob = lr.predict_proba(x_test)
prob

array([[0.8411099 , 0.1588901 ],
       [0.62772238, 0.37227762],
       [0.2957478 , 0.7042522 ],
       [0.89889359, 0.10110641],
       [0.25118347, 0.74881653],
       [0.55697283, 0.44302717],
       [0.60777912, 0.39222088],
       [0.92733973, 0.07266027],
       [0.2852556 , 0.7147444 ],
       [0.9870155 , 0.0129845 ],
       [0.63820175, 0.36179825],
       [0.47237823, 0.52762177],
       [0.04171002, 0.95828998],
       [0.15169917, 0.84830083],
       [0.14891193, 0.85108807],
       [0.19328429, 0.80671571],
       [0.09135474, 0.90864526],
       [0.80057316, 0.19942684],
       [0.95656707, 0.04343293],
       [0.93176677, 0.06823323],
       [0.87943286, 0.12056714],
       [0.95901625, 0.04098375],
       [0.67754065, 0.32245935],
       [0.11611361, 0.88388639],
       [0.23467626, 0.76532374],
       [0.01559839, 0.98440161],
       [0.11465776, 0.88534224],
       [0.62216329, 0.37783671],
       [0.59620847, 0.40379153],
       [0.05102426, 0.94897574],
       [0.

In [ ]:
#Predicting values
y_pred = lr.predict(x_test)

### Evaluation of model

In [ ]:
cfm = confusion_matrix(y_test,y_pred)
print("Confusion Matrix: ")
print(cfm)

acc = accuracy_score(y_test,y_pred)
print("accuracy score",acc)

print('classification report : ')
print(classification_report(y_test,y_pred))

Confusion Matrix: 
[[134  27]
 [ 23 131]]
accuracy score 0.8412698412698413
classification report : 
              precision    recall  f1-score   support

           0       0.85      0.83      0.84       161
           1       0.83      0.85      0.84       154

    accuracy                           0.84       315
   macro avg       0.84      0.84      0.84       315
weighted avg       0.84      0.84      0.84       315



### Building RandomForestModel

In [ ]:
rf = RandomForestClassifier(n_estimators=300, random_state=42)
rf.fit(x_train, y_train)

RandomForestClassifier(n_estimators=300, random_state=42)

In [ ]:
# Prediction on test data

y_pred1 = rf.predict(x_test)

In [ ]:
cfm = confusion_matrix(y_test,y_pred1)
print("Confusion Matrix: ")
print(cfm)

acc = accuracy_score(y_test,y_pred1)
print("accuracy score",acc)

print('classification report : ')
print(classification_report(y_test,y_pred1))

Confusion Matrix: 
[[137  24]
 [ 23 131]]
accuracy score 0.8507936507936508
classification report : 
              precision    recall  f1-score   support

           0       0.86      0.85      0.85       161
           1       0.85      0.85      0.85       154

    accuracy                           0.85       315
   macro avg       0.85      0.85      0.85       315
weighted avg       0.85      0.85      0.85       315



### Building XGBoost classifier model

In [ ]:
# Train XGBoost Model
xgb_model = XGBClassifier(n_estimators=250, learning_rate=0.1, max_depth=10, random_state=42)

xgb_model.fit(x_train, y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.1, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=10, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=250, n_jobs=None,
              num_parallel_tree=None, random_state=42, ...)

In [ ]:
y_pred2 = xgb_model.predict(x_test)

In [ ]:
cfm = confusion_matrix(y_test,y_pred2)
print("Confusion Matrix: ")
print(cfm)

acc = accuracy_score(y_test,y_pred2)
print("accuracy score",acc)

print('classification report : ')
print(classification_report(y_test,y_pred2))

Confusion Matrix: 
[[138  23]
 [ 27 127]]
accuracy score 0.8412698412698413
classification report : 
              precision    recall  f1-score   support

           0       0.84      0.86      0.85       161
           1       0.85      0.82      0.84       154

    accuracy                           0.84       315
   macro avg       0.84      0.84      0.84       315
weighted avg       0.84      0.84      0.84       315



In [ ]:
prob_test = xgb_model.predict_proba(df_test)
prob_test

array([[9.66391385e-01, 3.36086191e-02],
       [5.07874131e-01, 4.92125839e-01],
       [9.48501468e-01, 5.14985323e-02],
       [9.98950481e-01, 1.04949810e-03],
       [9.98889744e-01, 1.11025583e-03],
       [9.74470913e-01, 2.55290922e-02],
       [6.48919165e-01, 3.51080835e-01],
       [9.99347329e-01, 6.52687741e-04],
       [9.99732614e-01, 2.67383642e-04],
       [9.76316512e-01, 2.36834902e-02],
       [3.96050870e-01, 6.03949130e-01],
       [9.91323054e-01, 8.67692474e-03],
       [7.41749644e-01, 2.58250356e-01],
       [9.96688008e-01, 3.31198960e-03],
       [9.88211334e-01, 1.17886923e-02],
       [9.94965076e-01, 5.03493100e-03],
       [9.23159838e-01, 7.68401772e-02],
       [9.98390555e-01, 1.60946976e-03],
       [9.99053419e-01, 9.46552900e-04],
       [9.93890524e-01, 6.10946817e-03],
       [9.98037696e-01, 1.96232530e-03],
       [8.47334325e-01, 1.52665690e-01],
       [9.28878248e-01, 7.11217597e-02],
       [4.68253255e-01, 5.31746745e-01],
       [9.987015

In [ ]:
pred_test =[1 if i>0.44 else 0 for i in prob_test[:,1]]

In [ ]:
output = pd.DataFrame()
output['treatment'] = pred_test
output['treatment'] = output['treatment'].map({1:"Yes",0:"No"})

In [ ]:
print(output)

    treatment
0          No
1         Yes
2          No
3          No
4          No
..        ...
205        No
206        No
207       Yes
208        No
209       Yes

[210 rows x 1 columns]


In [ ]:
predictions_df = pd.DataFrame({'S.No': range(1, 211), 'treatment': output['treatment']})

In [ ]:
file_path = "/content/drive/MyDrive/Hackathon/XGB_Prediction.csv"

In [ ]:
predictions_df.to_csv(file_path, index=False)
print(f"File saved at: {file_path}")

File saved at: /content/drive/MyDrive/Hackathon/XGB_Prediction.csv
